# Strings and Representations

## Functions for making strings representations

1. `str()` - method `__str__()`
2. `repr()` - method `__repr__()`

In [2]:
class Point2D:
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __str__(self):
        return '(%s, %s)' % (self.x, self.y)

    def __repr__(self):
        return 'Point2D(x=%s, y=%s)' % (self.x, self.y)

In [2]:
point = Point2D(x=42, y=69)

In [3]:
str(point)

'(42, 69)'

In [4]:
repr(point)

'Point2D(x=42, y=69)'

## `repr()`

Produces an unambiguous string representation of an object.

- Exactness is more important than human-friendliness
- Suited for debugging
- Includes identifying information
- Best option for logging
- Should generally contains more information than result of `str()`
- pdb uses `repr()`

## `str()`

Produces a readable, human-friendly representation of an object

## When are the Representations Used

### `print`

`print` uses `str()`:

In [6]:
print(point)

(42, 69)


By default `str()` simply calls `repr()`. But `repr()` does not used `str()`.

### Collection Elements

`repr()` used when showing elements of a collection.

In [9]:
points = [Point2D(i, i*2) for i in range(3)]
str(points)

'[Point2D(x=0, y=0), Point2D(x=1, y=2), Point2D(x=2, y=4)]'

In [10]:
repr(points)

'[Point2D(x=0, y=0), Point2D(x=1, y=2), Point2D(x=2, y=4)]'

In [11]:
points_dict = {i: Point2D(i, i*2) for i in range(3)}
str(points_dict)

'{0: Point2D(x=0, y=0), 1: Point2D(x=1, y=2), 2: Point2D(x=2, y=4)}'

In [12]:
repr(points_dict)

'{0: Point2D(x=0, y=0), 1: Point2D(x=1, y=2), 2: Point2D(x=2, y=4)}'

### String Formatting

In [13]:
'This is a point: %s' % point

'This is a point: (42, 69)'

In [21]:
class Point2DFormatted(Point2D):
   
    def __format__(self, f):
        return '[Formatted point: %s, %s, %s]' % (self.x, self.y, f)

In [22]:
'This is a point: {}'.format(Point2DFormatted(1, 2))

'This is a point: [Formatted point: 1, 2, ]'

`f` stands for *format specification*

In [26]:
class Point2DFormattedReversable(Point2D):
    
    def __format__(self, f):
        if f == 'r':
            return '{}, {}'.format(self.y, self.x)
        else:
            return '{}, {}'.format(self.x, self.y, f)

In [29]:
reversable_point = Point2DFormattedReversable(1, 2)
'{}'.format(reversable_point)

'1, 2'

In [30]:
'{:r}'.format(reversable_point)

'2, 1'

By default `__format__()` just calls `__str__()`. To force using `__repr__()`:

In [31]:
'{!r}'.format(reversable_point)

'Point2D(x=1, y=2)'

`{!s}` forces the use of `__str__()`.

## `reprlib` module

Support alternative implementations of `repr()`.

- Limits otherwise excessive string length
- Useful for large collections

In [3]:
import reprlib
points = [Point2D(x, y) for x in range(1000) for y in range(1000)]
len(points)

1000000

In [4]:
reprlib.repr(points)

'[Point2D(x=0, y=0), Point2D(x=0, y=1), Point2D(x=0, y=2), Point2D(x=0, y=3), Point2D(x=0, y=4), Point2D(x=0, y=5), ...]'

### `reprlib.Repr` class

- Implements the main functionality of `reprlib`
- Supports customisation through subclassing
- Default behaviour can be changed with `reprlib.aRepr`

## `ascii()`, `ord()`, and `chr()`

### `ascii()`

Replaces non-ASCII characters with escape sequences

In [5]:
text = 'Hello мир'
type(text)

str

In [7]:
ascii_text = ascii(text)
ascii_text

"'Hello \\u043c\\u0438\\u0440'"

In [8]:
type(ascii_text)

str

### `ord()`

Converts a single character to its integer Unicode codepoint

In [9]:
char = 'Щ'
ord(char)

1065

### `chr()`

Converts an integer Unicode codepoint to a single character string

In [10]:
chr(1065)

'Щ'

## Bigger Isn't Always Better

In [38]:
class Table:
    
    def __init__(self, header, *data):
        self.header = header
        self.data = data
        assert len(header) == len(data)
    
    def _column_width(self, i):
        result = max(len(str(x)) for x in self.data[i])
        return max(len(self.header[i]), result)
    
    def __str__(self):
        col_count = len(self.header)
        col_widths = [self._column_width(i) for i in range(col_count)]
        format_specs = ['{{:{}}}'.format(col_widths[i]) for i in range(col_count)]
        
        result = []
        result.append((format_specs[i].format(self.header[i]) for i in range(col_count)))
        result.append(('='*col_widths[i] for i in range(col_count)))
        for row in zip(*self.data):
            result.append([format_specs[i].format(row[i]) for i in range(col_count)])
        result = (' '.join(r) for r in result)
        return '\n'.join(result)
    
    def __repr__(self):
        return 'Table(header={})'.format(self.header)

In [39]:
table = Table(['First name', 'Last name'],
              ['Fred', 'George', 'Scooby'],
              ['Flintstone', 'Jetson', 'Doo'])
print(table)

First name Last name 
========== ==========
Fred       Flintstone
George     Jetson    
Scooby     Doo       


In [40]:
repr(table)

"Table(header=['First name', 'Last name'])"